# Laboratories

AOC Day 7 Part 1

This feels like a mixture of matrices and trees/graphs. Let's bring in our `Matrix` and `Neighbours` implementations from Day 4. I have updated it a bit to fit the data types in the current problem.

In [145]:
class Neighbours:
    def __init__(self, a: str | None, b: str | None, c: str | None, m: str | None, n: str | None, p: str | None, q: str | None, r: str | None) -> None:
        self.a = a
        self.b = b
        self.c = c
        self.m = m
        self.n = n
        self.p = p
        self.q = q
        self.r = r

    # This function returns what's immediately above the current node
    # i.e., the North direction.
    @staticmethod
    def parent(matrix: "Matrix", of: tuple[int, int]) -> str | None:
        return Neighbours.of(matrix=matrix, node=of).b

    @staticmethod
    def of(matrix: "Matrix", node: tuple[int, int]):
        # x = rows, y = cols
        (x, y) = node
        x_min = True if x - 1 < 0 else False
        y_min = True if y - 1 < 0 else False

        x_max = True if x + 1 >= matrix.num_rows() else False
        y_max = True if y + 1 >= matrix.num_cols() else False

        a = None if (x_min or y_min) else matrix[x-1, y-1]
        b = None if x_min else matrix[x -1, y]
        c = None if (x_min or y_max) else matrix[x - 1, y + 1]
        m = None if y_min else matrix[x, y - 1]
        p = None if (y_min or x_max) else matrix[x + 1, y - 1]
        n = None if y_max else matrix[x, y + 1]
        r = None if x_max else matrix[x + 1, y]
        q = None if (y_max or x_max) else matrix[x + 1, y + 1]

        return Neighbours(
            a, b, c, m, n, p, q, r
        )
    
    def to_dict(self):
        return {
            "a": self.a,
            "b": self.b,
            "c": self.c,
            "m": self.m,
            "n": self.n,
            "p": self.p,
            "q": self.q,
            "r": self.r
        }
    
    def __iter__(self):
        yield from [
            self.a,
            self.b,
            self.c,
            self.m,
            self.n,
            self.p,
            self.r,
            self.q
        ]
        
class Matrix:
    def __init__(self, rows: list[str]) -> None:
        self.current_idx = -1
        self.last_idx = len(rows)
        self._vec: list[list] = []
        self._prepare_matrix(rows)
        pass

    def __iter__(self):
        return self
    
    def __next__(self):
        self.current_idx += 1
        if self.current_idx < self.last_idx:
            return self._vec[self.current_idx]

    def _prepare_matrix(self, rows: list[str]) -> None:
        self._vec = [["S" if ch == "S" else "^" if ch == "^" else "." for ch in line] for line in rows]
        
    def _neighbours(self, node: tuple[int, int]):
        return Neighbours.of(
            matrix=self,
            node=node
        )
    
    def __getitem__(self, key: tuple[int, int]):
        """
        key: (row, column)
        """
        (row, col) = key
        return self._vec[row][col]

    def __setitem__(self, key: tuple[int, int], value: str):
        """
        key: (row, column)
        """
        (row, col) = key
        self._vec[row][col] = value

    def __len__(self):
        return len(self._vec)
    
    def row(self, idx: int) -> list[str]:
        return self._vec[idx]

    def num_rows(self) -> int:
        return len(self._vec)
    
    def num_cols(self) -> int:
        return len(self._vec[0])
    
    def to_string(self) -> str:
        return str(self._vec)
    
    def pretty_print(self) -> None:
        for row in self._vec:
            print(''.join(row))
    
    def dim(self) -> tuple[int, int]:
        return (self.num_rows(), self.num_cols())
    
    def apply_row(self, n, f):
        # Apply function f to an entire row.
        N = self._vec[n]
        for i in range(len(N)):
            f(self, N[i])
    
    def apply(self, f, row_major=True):
        if row_major:
            for i in range(len(self._vec)):
                for j in range(len(self._vec[0])):
                    f(self, self[i, j])
        else:
            for i in range(len(self._vec)):
                for j in range(len(self._vec[0])):
                    f(self, self[j, i])

    def save(self, filename: str):
        """
        Save the matrix to a file, one row per line.
        Each row is joined into a string.
        """
        with open(filename, 'w') as f:
            for row in self._vec:
                f.write(''.join(row) + '\n')


Given below is the input grid.

```
.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
```

Given below is the path of the tachyon beam.

```
.......S.......
.......|.......
......|^|......
......|.|......
.....|^|^|.....
.....|.|.|.....
....|^|^|^|....
....|.|.|.|....
...|^|^|||^|...
...|.|.|||.|...
..|^|^|||^|^|..
..|.|.|||.|.|..
.|^|||^||.||^|.
.|.|||.||.||.|.
|^|^|^|^|^|||^|
|.|.|.|.|.|||.|
```

We need to model the behaviour of the tachyon beam first in order to solve this problem.

Let us read the input first.



In [146]:
input = [
    ".......S.......",
    "...............",
    ".......^.......",
    "...............",
    "......^.^......",
    "...............",
    ".....^.^.^.....",
    "...............",
    "....^.^...^....",
    "...............",
    "...^.^...^.^...",
    "...............",
    "..^...^.....^..",
    "...............",
    ".^.^.^.^.^...^.",
    "..............."
]

If we use our `Matrix` implementation to read this into a matrix:

In [147]:
matrix = Matrix(rows=input)

In [148]:
matrix.pretty_print()

.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............


In [149]:
matrix.dim()

(16, 15)

Looks like the matrix implementation is okay.

We will add some utility functions into the `Matrix` class to traverse through and navigate the grid.

These functions are currently global for testing, but should be moved into the Matrix class after validation. Utility functions for beam navigation in the grid.

In [154]:
def initiate_split(matrix: Matrix):
    """
    Initiate the split from the starting point S.
    """
    pass

def split_beam(matrix: Matrix, row: int):
    """Placeholder for logic to split the beam on the given row in the matrix. n here is the row index."""
    N = matrix.row(row)
    for i in range(len(N)):
        # TODO: We need to check if the beam has actually arrived here.
        if row - 1 >= 0 and matrix[row - 1, i] == "|" and N[i] == "^":
            # split the beam.
            if i - 1 >= 0:
                matrix[row, i - 1] = "|"
            if i + 1 < len(N):
                matrix[row, i + 1] = "|"

matrix_cpy = Matrix(input)

for r in range(matrix_cpy.num_rows()):
    split_beam(matrix=matrix_cpy, row=r)
matrix_cpy.pretty_print()

matrix_cpy.save("matrix.txt")

.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............


In [151]:
def start_coordinates(vec: list[list[str]]) -> tuple[int, int]:
    """Return the (row, col) where the tachyon beam enters the matrix (the 'S' position in the top row)."""
    return (0, vec[0].index("S"))

In [152]:
def move(row: int, vec: list[list]) -> None:
    """Placeholder for logic to move the beam downwards through the y-axis by 1 step."""
    pass